In [1]:
from glob import glob
import json
import mwparserfromhell
from tqdm import trange
from transformers import AutoModel, AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForFeatureExtraction
from sentence_transformers import SentenceTransformer
import numpy as np

[2024-05-16 14:42:24,431] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
data = []
for filename in glob('../data/*/*.json'):
    lines = json.load(open(filename))
    data.extend(lines)

In [3]:
for i in trange(len(data)):
    data[i]["id"] = i
    if 'wikitext' in data[i]:
        data[i]['content'] = mwparserfromhell.parse(data[i]['wikitext']).strip_code().strip()

100%|██████████| 18739/18739 [00:30<00:00, 609.85it/s] 


In [4]:
# put into elastiscearch
from elasticsearch import Elasticsearch
es = Elasticsearch()
# es.indices.create(index='wikipedia', ignore=400)
for i in trange(len(data)):
    es.index(index='naruto_wiki', id=i, body=data[i])


100%|██████████| 18739/18739 [01:11<00:00, 261.25it/s]


In [2]:
# embeddings by nomic ai
# Use a pipeline as a high-level helper
model_name = "mixedbread-ai/mxbai-embed-large-v1"
model = SentenceTransformer(model_name, device="mps", )
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# ort_model = ORTModelForFeatureExtraction.from_pretrained(model_name, file_name="onnx/model_quantized.onnx")
# onnx_pipeline = pipeline("feature-extraction", model=ort_model, tokenizer=tokenizer)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [5]:
texts = [d['content'] for d in data]
batch_size = 256
# embeddings = np.zeros((len(texts), ort_model.config))

In [37]:
max_length = 512
# Split into chunks of max_length if length of input_ids is greater than max_length
# we mark chunk with index. Each index will have at least one chunk

chunks = []
for i in trange(0, len(data)):
    text = data[i]['content']
    if not text:
        continue
    inputs = model.tokenize(text)
    length = len(inputs["input_ids"])
    for j in range(length):
        for chunk in range(0, len(inputs["input_ids"][j]), max_length):
            chunks.append(
                (
                    data[i]["id"],
                    chunk,
                    {
                        "input_ids": inputs["input_ids"][j][
                            chunk : chunk + max_length
                        ],
                        "attention_mask": inputs["attention_mask"][j][
                            chunk : chunk + max_length
                        ],
                        "token_type_ids": inputs["token_type_ids"][j][
                            chunk : chunk + max_length
                        ],
                    },
                )
            )

 56%|█████▌    | 10480/18739 [1:01:32<02:39, 51.94it/s]  

: 

In [29]:
len(data)

18739

In [35]:
model.tokenize([data[0]["content"]])

{'input_ids': tensor([[  101,  1996,  2003,  2019,  6058,  4130,  6028,  2008,  4473,  1037,
          15544, 10087,  5289,  5310,  2000, 17708,  2039,  2000,  2416,  4230,
           2004,  2295,  2027,  2024,  2037,  2219,  1012,  8192,  1996,  5310,
           7861,  8270,  2015,  2028,  2030,  2062,  2304, 19278,  2046,  1037,
           2303,  1010,  4352,  2068,  2000,  3149,  2037, 15775, 22272,  2046,
           2009,  2013,  2307, 12103,  1012,  2000,  2191,  2440,  2224,  1997,
           1996,  2303,  1010,  1996,  5310, 28946, 19818,  2015,  2037, 15775,
          22272,  2013,  1996,  3284,  1998,  7541,  2391,  2825,  2061,  2008,
           2027,  2064,  2031,  1996,  2190,  2825,  2846,  1012,  6583, 22134,
           2080,  3127,  4413,  2620,  1010,  5530,  2403,  1011,  2321,  2320,
           2037, 15775, 22272,  8039,  1996,  2303,  1010,  2027,  2024,  2445,
           3143,  2491,  1997,  2009,  1010,  4352,  2068,  2000,  5630,  2035,
           1997,  2049,  45